In [ ]:
from exod.utils.path import data, data_raw
from exod.pre_processing.download_observations import read_observation_ids
from exod.pre_processing.read_events import read_EPIC_events_file, get_filtered_events_files, histogram_events_list

from glob import glob
import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
from scipy.stats import binned_statistic_dd
from astropy.io import fits
from itertools import combinations

In [ ]:
obsids = read_observation_ids(data / 'observations.txt')                                                  
obsid = obsids[4]
obsid = '0831790701'
obsid = '0112570701'
args = {'obsid':obsid,
        'size_arcsec':15,
        'time_interval':400,
        'box_size':3,
        'gti_only':True,
        'min_energy':0.2,
        'max_energy':12}

In [ ]:
%timeit fits.open('../data/processed/0001730201/P0001730201M1S001MIEVLI0000_FILT.fits')[1].header['TSTOP']

In [ ]:
%timeit fits.getheader('../data/processed/0001730201/P0001730201M1S001MIEVLI0000_FILT.fits', 1)['TSTOP']

In [ ]:
files = get_filtered_events_files(obsid)

In [ ]:
file_intervals

In [ ]:
or obsid in obsids:
    try:
        evt_subsets = get_overlapping_eventlist_subsets(obsid)
    except Exception as e:
        print(e)

In [ ]:
def plot_observation_times(obsid):
    files = get_filtered_events_files(obsid)
    plt.figure(figsize=(5,2))
    colors = [f'C{i}' for i in range(1,10)]
    file_intervals = {}
    for i, f in enumerate(files):
        hdul = fits.open(f)
        h = hdul[1].header
        TSTART, TSTOP = h['TSTART'], h['TSTOP']
        
        file_intervals[f] = [TSTART, TSTOP]
        
        tab = Table.read(f, hdu=1)
        t0, tf = tab['TIME'].min(), tab['TIME'].max()
        print(f'{TSTART:.2f}, {TSTOP:.2f}')
        print(f'{t0:.2f}, {tf:.2f}')
        print('====')
        plt.hlines(y=i, xmin=t0, xmax=tf, label=f'{f.stem}', color=colors[i])
        plt.hlines(y=i+0.5, xmin=TSTART, xmax=TSTOP, label=f'{f.stem}', color=colors[i])
    plt.title(obsid)
    plt.ylim(-1,len(files)+1)
    plt.legend(bbox_to_anchor=[0,0,0,1])
    plt.xlabel('Time (s)')    
    plt.show()

    grouped_intervals = group_overlapping_intervals(file_intervals)
    for k, v in grouped_intervals.items():
        print(k, v)
    print(grouped_intervals)

for obsid in obsids:
    try:
        plot_observation_times(obsid)
    except Exception as e:
        print(f'{e}')


In [ ]:
for obsid in obsids:
    try:
        files = get_filtered_events_files(obsid)
        for f in files:
            if 'PI' in f.stem:
                tab = Table.read(f)
                print(f)
                
    except FileNotFoundError:
        pass
        

In [ ]:
?binned_statistic_dd

In [ ]:
for obsid in obsids:
    print(obsid)
    obsid_path = data_raw / obsid
    try:
        files = get_filtered_events_files(obsid)
    except FileNotFoundError as e:
        print(f'{e}')
    for f in files:
        tab = Table.read(f, hdu=1)
        plt.figure(figsize=(5,5))
        plt.scatter(tab['X'], tab['Y'], s=0.0001)
        plt.show()
    print('========================')

In [ ]:
for obsid in obsids:
    print(obsid)
    obsid_path = data_raw / obsid
    files = list(obsid_path.glob('*.FTZ'))
    if not files:
        print('No .FTZ files found!')
        continue
    print(f'{len(files)} .FTZ files found')

    
    for i, f in enumerate(files):
        hdul = fits.open(f)
        submode = hdul[0].header['SUBMODE']
        tab = Table.read(f, hdu=1)
        ccdnrs = np.unique(tab['CCDNR'])
        N_ccds = len(ccdnrs)
        fig, ax = plt.subplots(1, N_ccds, figsize=(3*N_ccds, 3))
        try:
            for i, ccdnr in enumerate(ccdnrs):
                tab1 = tab[tab['CCDNR'] == ccdnr]
                fig.suptitle(f'{f.stem} {submode}')
                ax[i].set_facecolor('black')
                ax[i].scatter(tab1['RAWX'], tab1['RAWY'], c=tab1['PI'], s=0.01, marker='.', cmap='coolwarm_r')
        except TypeError:
            ax.scatter(tab1['RAWX'], tab1['RAWY'], s=0.001)
        plt.show()
        
    print('========================')

In [ ]:
# espfilt eventfile=P0675010401PNS003PIEVLI0000_FILT.fits
t1 = Table.read('../data/processed/0675010401/P0675010401PNS003PIEVLI0000_FILT.fits', hdu=1)
t2 = Table.read('../data/processed/0675010401/pnS003-allevc.fits', hdu=1)
t3 = Table.read('../data/processed/0675010401/pnS003-allimc.fits', hdu=1)
t4 = Table.read('../data/processed/0675010401/pnS003-gti.fits', hdu=1)
t4

In [ ]:
h1,b1 = histogram_events_list(t1, bin_size_seconds=10)
h2,b2 = histogram_events_list(t2, bin_size_seconds=10)
fig, ax = plt.subplots(2,1,figsize=(10,5), sharex=True)
ax[0].scatter(b1[:-1], h1, marker='.')
ax[1].scatter(b2[:-1], h2, marker='.')
for r in t3:
    ax[1].hlines(y=-10,xmin=r['START'], xmax=r['STOP'], color='red', lw=1.0)
for r in t4:
    ax[1].hlines(y=-10,xmin=r['START'], xmax=r['STOP'], color='cyan', lw=1.0)

In [ ]:
d1 = fits.open('../data/processed/0675010401/P0675010401PNS003PIEVLI0000_IMG.fits')[0].data
d2 = fits.open('../data/processed/0675010401/pnS003-allimc.fits')[0].data

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,7.5))
for a in ax:
    a.set_facecolor('black')
ax[0].set_title(f'Filtered Image sum:{np.sum(d1)}')
ax[0].imshow(d1, norm=LogNorm(), interpolation='none', cmap='hot')
ax[1].set_title(f'Filtered Image  + espfilt  sum:{np.sum(d2)}')
ax[1].imshow(d2, norm=LogNorm(), interpolation='none', cmap='hot', origin='lower')